In [1]:
import pandas as pd
import numpy as np
import random
import sys
import os

sys.path.append(os.path.abspath('..'))
from utils.db import get_db

In [2]:
engine = get_db()
start_date = '5/05/2025'
end_date = '6/23/2025'

#### Get the Query of base data for everything

In [ ]:
query = f"""SELECT 
    storeid,
    sale_date,
    product_code,
    price,
    cost,
    case_size,
    qty,
    total_sales
FROM sales
WHERE sale_date BETWEEN '{start_date}' and '{end_date}'
  AND store_number IN ('440')
"""

In [4]:
ds = pd.read_sql(query, engine)

#ensure it is datetime format

ds["sale_date"] = pd.to_datetime(ds["sale_date"])

2025-06-30 15:57:48,458 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-06-30 15:57:48,459 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-30 15:57:48,515 INFO sqlalchemy.engine.Engine select current_schema()
2025-06-30 15:57:48,516 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-30 15:57:48,571 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-06-30 15:57:48,571 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-06-30 15:57:48,630 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-06-30 15:57:48,631 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

In [5]:
ds = ds.dropna(subset=['price', 'cost', 'qty','case_size'])
ds = ds[ds['qty'] > 0]

ds["cost"] = (ds["cost"])/(ds["case_size"])
ds.head(10)

,storeid,sale_date,category,product_code,price,cost,case_size,qty,total_sales
0,1397,2025-05-25,999996.0,28500700000.0,4.81,0.010000,1.0,1.0,4.81
1,1397,2025-05-25,56000.0,28525900000.0,5.19,0.010000,1.0,1.0,5.19
2,1397,2025-05-25,56000.0,28525900000.0,3.54,0.010000,1.0,1.0,3.54
3,1397,2025-05-25,99995.0,28541300000.0,13.74,0.010000,1.0,1.0,13.74
4,1397,2025-05-25,99995.0,28541300000.0,11.23,0.010000,1.0,1.0,11.23
5,1397,2025-05-25,99995.0,28541300000.0,4.87,0.010000,1.0,1.0,4.87
6,1397,2025-05-24,24003.0,7003859100.0,4.69,2.501667,12.0,1.0,4.69
7,1397,2025-05-24,91201.0,4660000090.0,2.79,1.618333,12.0,1.0,0.99
8,1397,2025-05-24,4001.0,7062243101.0,0.59,0.260000,36.0,1.0,0.59
9,1397,2025-05-24,55001.0,7273033110.0,3.69,2.500000,1.0,1.0,3.69


##### Set Day for periods, monday

In [6]:
ds['week_start'] = ds['sale_date'] - pd.to_timedelta(ds['sale_date'].dt.dayofweek, unit='d') 


#### Weekly Sales

In [11]:
# Group by week, storeid, product_code to get weekly sales
weekly_sales = ds.groupby(['storeid', 'product_code', 'week_start']).agg(
    current_week_sales=('price', 'sum')
).reset_index()

In [12]:
weekly_sales.head(10)

,storeid,product_code,week_start,current_week_sales
0,1397,1018677323.0,2025-05-12,9.99
1,1397,1022810403.0,2025-05-12,3.00
2,1397,1022810403.0,2025-05-26,3.00
3,1397,1022810404.0,2025-05-05,5.00
4,1397,1022810404.0,2025-06-02,10.00
5,1397,1037416200.0,2025-05-05,7.99
6,1397,1037416200.0,2025-05-19,15.98
7,1397,1037416200.0,2025-06-02,7.99
8,1397,1037416200.0,2025-06-09,7.99
9,1397,1037416200.0,2025-06-16,7.99


#### Lets get Lagging ...

In [13]:
# Sort to prepare for lag comparison
weekly_sales = weekly_sales.sort_values(by=['storeid', 'product_code', 'week_start'])

##### We got to get periods, so lets use shift

In [14]:
weekly_sales['prev_week_1_sales'] = weekly_sales.groupby(['storeid', 'product_code'])['current_week_sales'].shift(1)
weekly_sales['prev_week_2_sales'] = weekly_sales.groupby(['storeid', 'product_code'])['current_week_sales'].shift(2)


In [15]:
# Replace NaNs in lag columns with 0.00
weekly_sales[['prev_week_1_sales', 'prev_week_2_sales']] = weekly_sales[['prev_week_1_sales', 'prev_week_2_sales']].fillna(0.00)

# Calculate percentage movement (handle divide-by-zero safely)
weekly_sales['movement_vs_prev_week_1'] = weekly_sales.apply(
    lambda row: ((row['current_week_sales'] - row['prev_week_1_sales']) / row['prev_week_1_sales'] * 100)
    if row['prev_week_1_sales'] != 0 else 0.00,
    axis=1
)

weekly_sales['movement_vs_prev_week_2'] = weekly_sales.apply(
    lambda row: ((row['current_week_sales'] - row['prev_week_2_sales']) / row['prev_week_2_sales'] * 100)
    if row['prev_week_2_sales'] != 0 else 0.00,
    axis=1
)

In [16]:
# Optional: round for neatness
weekly_sales = weekly_sales.round(2)

# Preview
print(weekly_sales.head(10))

   storeid  product_code week_start  current_week_sales  prev_week_1_sales  \
0     1397  1018677323.0 2025-05-12                9.99               0.00   
1     1397  1022810403.0 2025-05-12                3.00               0.00   
2     1397  1022810403.0 2025-05-26                3.00               3.00   
3     1397  1022810404.0 2025-05-05                5.00               0.00   
4     1397  1022810404.0 2025-06-02               10.00               5.00   
5     1397  1037416200.0 2025-05-05                7.99               0.00   
6     1397  1037416200.0 2025-05-19               15.98               7.99   
7     1397  1037416200.0 2025-06-02                7.99              15.98   
8     1397  1037416200.0 2025-06-09                7.99               7.99   
9     1397  1037416200.0 2025-06-16                7.99               7.99   

   prev_week_2_sales  movement_vs_prev_week_1  movement_vs_prev_week_2  
0               0.00                      0.0                      0

#### Define our Summary of the data and previous data totals

In [17]:
summary = weekly_sales.groupby(['storeid', 'product_code']).agg(
    total_sales=('current_week_sales', 'sum'),
    total_prev_week_1_sales=('prev_week_1_sales', 'sum'),
    total_prev_week_2_sales=('prev_week_2_sales', 'sum'),
    avg_movement_vs_prev_week_1=('movement_vs_prev_week_1', 'mean'),
    avg_movement_vs_prev_week_2=('movement_vs_prev_week_2', 'mean'),
    total_weeks=('week_start', 'count')  # Optional, to see how many weeks data exists
).reset_index()


In [18]:
summary = summary.round(2)
print(summary.head(10))

   storeid  product_code  total_sales  total_prev_week_1_sales  \
0     1397  1018677323.0         9.99                     0.00   
1     1397  1022810403.0         6.00                     3.00   
2     1397  1022810404.0        15.00                     5.00   
3     1397  1037416200.0        47.94                    39.95   
4     1397  1037416201.0        63.92                    39.95   
5     1397  1037416205.0       111.86                    63.92   
6     1397  1037416270.0        23.97                    15.98   
7     1397  1037422282.0        53.91                    47.92   
8     1397  1037422284.0        11.98                     5.99   
9     1397  1037422285.0        47.92                    35.94   

   total_prev_week_2_sales  avg_movement_vs_prev_week_1  \
0                     0.00                         0.00   
1                     0.00                         0.00   
2                     0.00                        50.00   
3                    31.96           

In [19]:
print(summary[['product_code', 'total_sales', 'total_prev_week_1_sales', 
               'total_prev_week_2_sales', 'total_weeks']].head(10).to_string(index=False))

product_code  total_sales  total_prev_week_1_sales  total_prev_week_2_sales  total_weeks
1018677323.0         9.99                     0.00                     0.00            1
1022810403.0         6.00                     3.00                     0.00            2
1022810404.0        15.00                     5.00                     0.00            2
1037416200.0        47.94                    39.95                    31.96            5
1037416201.0        63.92                    39.95                    23.97            4
1037416205.0       111.86                    63.92                    47.94            6
1037416270.0        23.97                    15.98                     0.00            2
1037422282.0        53.91                    47.92                    41.93            5
1037422284.0        11.98                     5.99                     0.00            2
1037422285.0        47.92                    35.94                    23.96            5


#### This is the magic sauce right here. Need to take the spread of the data somehow for sales trend of rise or fall.  with Mike lol


###### If: W-2 <  W-1  < Current = Rise
###### If: W-2 >  W-1  >Current = Faling
###### If: W-2 =< W-1  >=Current = Stable
###### IF: W-2 => W-1  <=Current = Stable 
###### Else: = Stable

###### This continues to be fun, we made logic and tested by the results did not turn out correct. So the suggestion is to use the and word instead of linking 3 conditions

In [20]:
def classify_sales_trend(row):
    w2 = row['total_prev_week_2_sales']
    w1 = row['total_prev_week_1_sales']
    current = row['total_sales']
    
    if w1 >= current:
        return 'Falling'
    elif  w2 < w1 < current :
        return 'Rising'
    elif w2 <= w1 >= current:
        return 'Close'
    elif w2 >= w1 <= current:
        return 'Almost'
    else:
        return 'Other'
    



In [21]:
# Make sure no NaNs exist in these columns
summary[['total_prev_week_2_sales', 'total_prev_week_1_sales', 'total_sales']] = summary[
    ['total_prev_week_2_sales', 'total_prev_week_1_sales', 'total_sales']
].fillna(0.00)


# Apply classify_sales_trend classification
summary['sales_trend'] = summary.apply(classify_sales_trend, axis=1)


##### Debug this thing becuase we arent getting it still

In [22]:
summary['debug_tuple'] = summary[['total_prev_week_2_sales', 'total_prev_week_1_sales', 'total_sales']].apply(
    lambda x: f"W2={x[0]} W1={x[1]} Cur={x[2]}", axis=1
)

print(summary[['product_code', 'debug_tuple', 'sales_trend']].head(10))

C:\Users\tadaur\AppData\Local\Temp\ipykernel_39376\2953675238.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lambda x: f"W2={x[0]} W1={x[1]} Cur={x[2]}", axis=1


   product_code                   debug_tuple sales_trend
0  1018677323.0        W2=0.0 W1=0.0 Cur=9.99      Almost
1  1022810403.0         W2=0.0 W1=3.0 Cur=6.0      Rising
2  1022810404.0        W2=0.0 W1=5.0 Cur=15.0      Rising
3  1037416200.0   W2=31.96 W1=39.95 Cur=47.94      Rising
4  1037416201.0   W2=23.97 W1=39.95 Cur=63.92      Rising
5  1037416205.0  W2=47.94 W1=63.92 Cur=111.86      Rising
6  1037416270.0     W2=0.0 W1=15.98 Cur=23.97      Rising
7  1037422282.0   W2=41.93 W1=47.92 Cur=53.91      Rising
8  1037422284.0      W2=0.0 W1=5.99 Cur=11.98      Rising
9  1037422285.0   W2=23.96 W1=35.94 Cur=47.92      Rising


##### Now we need to count the Sales trend Types

In [23]:
trend_summary = summary['sales_trend'].value_counts().reset_index()
trend_summary.columns = ['Sales Trend', 'Count']

print("=== Trend Summary ===")
print(trend_summary)

=== Trend Summary ===
  Sales Trend  Count
0      Rising  12274
1      Almost   4542


##### We are getting any "Falling'  Sales_trends for some reason.. not sure if I believe that

In [24]:
print(summary[['product_code', 'total_sales', 'total_prev_week_1_sales', 
               'total_prev_week_2_sales', 'total_weeks', 'sales_trend']].head(10).to_string(index=False))

product_code  total_sales  total_prev_week_1_sales  total_prev_week_2_sales  total_weeks sales_trend
1018677323.0         9.99                     0.00                     0.00            1      Almost
1022810403.0         6.00                     3.00                     0.00            2      Rising
1022810404.0        15.00                     5.00                     0.00            2      Rising
1037416200.0        47.94                    39.95                    31.96            5      Rising
1037416201.0        63.92                    39.95                    23.97            4      Rising
1037416205.0       111.86                    63.92                    47.94            6      Rising
1037416270.0        23.97                    15.98                     0.00            2      Rising
1037422282.0        53.91                    47.92                    41.93            5      Rising
1037422284.0        11.98                     5.99                     0.00            2   

#### You got Trends right here for 3 weeks comparison for all itesm.  Lets try to get it grouped and assigned with only what we want 

#### This are differnt export types, it can take all of them together and we can move throught the data.

In [ ]:
# Sales Trend Summary
summary_subset = summary[['product_code', 'total_sales', 'total_prev_week_1_sales', 
                          'total_prev_week_2_sales', 'total_weeks', 'sales_trend']]

summary_subset.to_csv("summary_by_product.csv", index=False)

In [27]:
# Add Percent Change Columns 

summary['pct_change_w2_to_w1'] = (
    (summary['total_prev_week_1_sales'] - summary['total_prev_week_2_sales']) /
    summary['total_prev_week_2_sales'].replace(0, np.nan)
) * 100

summary['pct_change_w1_to_current'] = (
    (summary['total_sales'] - summary['total_prev_week_1_sales']) /
    summary['total_prev_week_1_sales'].replace(0, np.nan)
) * 100

In [ ]:
summary['avg_trend_score'] = (
    (summary['pct_change_w2_to_w1'] + summary['pct_change_w1_to_current']) / 2
)

In [28]:
summary_subset = summary[['product_code', 'total_sales', 'total_prev_week_1_sales',
                          'total_prev_week_2_sales', 'total_weeks', 'sales_trend',
                          'pct_change_w2_to_w1', 'pct_change_w1_to_current']]

summary_subset.to_csv("summary_by_product_with_percent_change.csv", index=False)

##### Might be bad 6/29 4:14 and below

In [85]:
# Helper: get the min and max sale dates per product/store
date_range = weekly_sales.groupby(['product_code']).agg(
    start_date=('week_start', 'min'),
    end_date=('week_start', 'max')
).reset_index()

In [86]:
# Merge trend labels and date range back into summary
# Group by what you want
raw_grouped = weekly_sales.groupby([ 'product_code', 'sales_trend']).agg(
    total_sales=('current_week_sales', 'sum'),
    total_prev_week_1_sales=('prev_week_1_sales', 'sum'),
    total_prev_week_2_sales=('prev_week_2_sales', 'sum'),
    avg_movement_vs_prev_week_1=('movement_vs_prev_week_1', 'mean'),
    avg_movement_vs_prev_week_2=('movement_vs_prev_week_2', 'mean'),
    week_count=('week_start', 'count')
).reset_index()

# Merge in the date range
raw_grouped = raw_grouped.merge(date_range, on=['product_code'])

In [87]:
# Optional: format the date range as a single column
raw_grouped['date_range'] = raw_grouped['start_date'].dt.strftime('%Y-%m-%d') + " to " + raw_grouped['end_date'].dt.strftime('%Y-%m-%d')
raw_grouped = raw_grouped.drop(columns=['start_date', 'end_date'])  # drop separate date cols if not needed

# Split by sales trend
rising_grouped = raw_grouped[raw_grouped['sales_trend'] == 'Rising']
falling_grouped = raw_grouped[raw_grouped['sales_trend'] == 'Falling']
stable_grouped = raw_grouped[raw_grouped['sales_trend'] == 'Stable']

In [88]:
rising_grouped.head(10)

,product_code,sales_trend,total_sales,total_prev_week_1_sales,total_prev_week_2_sales,avg_movement_vs_prev_week_1,avg_movement_vs_prev_week_2,week_count,date_range
2,1022810404.0,Rising,15.00,5.00,0.00,50.000000,0.000000,2,2025-05-05 to 2025-06-02
3,1037416200.0,Rising,47.94,39.95,31.96,10.000000,-10.000000,5,2025-05-05 to 2025-06-16
4,1037416201.0,Rising,63.92,39.95,23.97,25.000000,50.000000,4,2025-05-05 to 2025-06-09
5,1037416205.0,Rising,111.86,63.92,47.94,38.888333,88.888333,6,2025-05-05 to 2025-06-09
7,1037422282.0,Rising,53.91,47.92,41.93,26.666000,13.332000,5,2025-04-28 to 2025-06-09
9,1037422285.0,Rising,47.92,35.94,23.96,20.000000,40.000000,5,2025-05-05 to 2025-06-09
10,1037442060.0,Rising,154.90,122.92,106.93,3.335000,-1.850000,6,2025-05-05 to 2025-06-09
11,1037442063.0,Rising,59.96,14.99,0.00,100.000000,0.000000,2,2025-05-12 to 2025-05-26
14,1066807502.0,Rising,77.85,43.25,34.60,108.332500,8.332500,4,2025-04-28 to 2025-06-02
16,1066807519.0,Rising,60.55,51.90,34.60,29.167500,8.332500,4,2025-04-28 to 2025-06-16


In [90]:
from sklearn.preprocessing import MinMaxScaler

# Create a working copy of the grouped data
scalable_df = raw_grouped.copy()

# Optional: Replace inf/nan if any anomalies remain
scalable_df['avg_movement_vs_prev_week_1'] = scalable_df['avg_movement_vs_prev_week_1'].replace([np.inf, -np.inf], np.nan).fillna(0)

# Initialize the scaler
scaler = MinMaxScaler(feature_range=(-1, 1))  # Use (0, 1) if you want only positive values

# Fit and transform based on avg_movement
scalable_df['growth_score'] = scaler.fit_transform(
    scalable_df[['avg_movement_vs_prev_week_1']]
)

# Merge scaled score back into trend-specific DataFrames
rising_grouped = rising_grouped.merge(
    scalable_df[['product_code', 'growth_score']],
    on='product_code'
)

falling_grouped = falling_grouped.merge(
    scalable_df[['product_code', 'growth_score']],
    on='product_code'
)

stable_grouped = stable_grouped.merge(
    scalable_df[['product_code', 'growth_score']],
    on='product_code'
)

In [92]:
rising_grouped.to_csv("rising_grouped_raw_records.csv", index=False)
falling_grouped.to_csv("falling_grouped_raw_records.csv", index=False)
stable_grouped.to_csv("stable_groupd_raw_records.csv", index=False)